In [2]:
from satellite import Satellite

sat = Satellite(
    sma=6878,
    ecc=0.001,
    inc=97.5
)

states = sat.propagate(duration_sec=5400)

states.shape

APIException: ODEModel Exception Thrown: Attempted to add a GravityField (aka primary body) force to the force model "FM" for the body Earth, but there already is a Gravity Field in the ODE Model and only one GravityField is supported per ODE Model in the current GMAT release